In [68]:
import pandas as pd
import glob
import re

> Load all the data into one dataframe

In [64]:
# Using glob  & regex to get all the files appropriate files in the directory
egrid_files = glob.glob("../data/rmr_62/egrid_data/e[Gg][Rr][Ii][Dd]*.xls*")

In [72]:
# Reading the egrid files into a pandas dataframe, only keep the tab we need
egrid_df = pd.DataFrame()
for file in egrid_files:
    match = re.search(r"\d{4}", file)
    year = match.group()

    df = pd.read_excel(file, sheet_name=f"PLNT{year[-2]}{year[-1]}", skiprows=1)
    df["Year"] = year

    egrid_df = pd.concat([egrid_df, df])

del df  # Clean up


## Column name descriptions

| Column    | Description |
|-----------|-------------|
| PNAME     | Plan Name |
| PLPRMFL   | Primary Fuel |
| CAPFAC    | Capacity Facoty |
| NAMEPCAP  | Nameplate Capacity |
| PLNGENAN  | Plant annual net genration (MWh) |
| PLNOXAN   | Plant annual Nox Emission (tonnes) |
| PLSO2AN   | Plant annual So2 Emission (tonnes) |
| PLCO2AN   | Plant annual Co2 Emission (tonnes) |
| Year      | Data for Year |
| TotEm     | Total aanual plant emissions (tonnes) |

In [78]:
egrid_df = egrid_df[
    [
        "PNAME",
        "PLPRMFL",
        "CAPFAC",
        "NAMEPCAP",
        "PLNGENAN",
        "PLNOXAN",
        "PLSO2AN",
        "PLCO2AN",
        "Year",
    ]
]

c = "TotEm"
egrid_df[c] = egrid_df["PLNOXAN"] + egrid_df["PLSO2AN"] + egrid_df["PLCO2AN"]

egrid_df

,PNAME,PLPRMFL,CAPFAC,NAMEPCAP,PLNGENAN,PLNOXAN,PLSO2AN,PLCO2AN,Year,TotEm
0,Agrium Kenai Nitrogen Operations,NG,0.000000,21.6,0.00,0.000000,0.000000,0.000000e+00,2012,0.000000e+00
1,Alakanuk,JF,0.178346,1.3,2031.00,21.044206,0.091765,1.374568e+03,2012,1.395704e+03
2,Anchorage 1,NG,0.060609,102.9,54633.00,95.333378,1.242881,4.038460e+04,2012,4.048117e+04
3,Angoon,DFO,0.134323,1.5,1765.00,18.288047,1.311343,1.223040e+03,2012,1.242640e+03
4,Aniak,DFO,0.286322,1.1,2759.00,33.331117,2.519999,2.229068e+03,2012,2.264919e+03
...,...,...,...,...,...,...,...,...,...,...
4993,Spirit Mountain,WAT,0.335000,4.5,13204.00,0.000000,0.000000,0.000000e+00,2005,0.000000e+00
4994,Strawberry Creek,WAT,0.742100,1.5,9750.95,0.000000,0.000000,0.000000e+00,2005,0.000000e+00
4995,Wygen 1,SUB,0.919900,88.0,709116.89,561.491000,538.381000,1.040668e+06,2005,1.041768e+06
4996,Wyodak,SUB,0.843700,362.0,2675359.00,4946.278000,7732.067000,3.370621e+06,2005,3.383299e+06


In [60]:
emissions_df = pd.read_csv("../data/rmr_62/emissions_data/emissions_epa.csv")

In [61]:
emissions_df

,Year,Carbon dioxide,Methane,Nitrous oxide,"HFCs, PFCs, SF6, and NF3"
0,1990,"5,120.96",868.661,406.281,91.432
1,1991,"5,062.96",874.355,397.595,83.500
2,1992,"5,174.23",874.450,398.820,87.383
3,1993,"5,272.85",863.427,415.889,87.358
4,1994,"5,364.28",871.454,418.494,90.940
5,1995,"5,425.26",865.942,422.923,107.809
6,1996,"5,612.98",859.731,432.744,118.072
7,1997,"5,688.14",845.140,421.073,124.851
8,1998,"5,733.28",830.142,422.535,138.727
9,1999,"5,803.92",812.532,416.449,136.987
